<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=fd4eb8e8d3af86c5b7813c607932ec4761c7b15abd086dca343054fac4447227
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-27 15:06:11
-------------------
qualified stocks: 89
with latest results: 28
still star stocks: 17
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  2.00 K
Current:  1.45 C
-------------------
Today PnL: 63.74 K (0.44%)
Current PnL: -19.10 L (-12.52%)
CY Booked + Current PnL: -7.00 L (-4.59%)
-------------------
Total profit:  2.51 L
Total loss:  -21.61 L
-------------------
Total Booked + Current PnL: 19.98 L (15.9%)
Total Booked PnL: 39.08 L (31.11%)
Curr Year Booked PnL: 12.10 L (8.34%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 87.00 L (59.97%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.6%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RELIANCE,1533.0,-7.66,72.0,X-LC,3.51,231426.0,21060.0,7730.0,2.20,10.01,3.34,13.68,37.0,2.72,1.60,28.19,XY25,NTT,REFINERIES
86,VOLTAS,1530.0,2.56,58.0,X-MC,3.10,215070.0,23328.0,14431.0,-0.53,12.17,6.71,19.69,99.0,1.62,1.48,19.79,XY25,NTT,AC
42,ITC,452.0,-36.23,67.0,X-LC,1.71,205380.0,5242.0,15650.0,0.77,2.62,7.62,10.44,4.0,0.33,1.42,8.33,X40,NTT,FMCG
79,TTKPRESTIG,770.0,97.23,52.0,M-SC,2.06,84947.0,-15830.0,15919.0,0.55,-15.71,18.74,0.09,245.0,-0.99,0.59,11.60,OX40N,NTT,DURABLES
80,UJJIVANSFB,60.0,131.04,74.0,H-SC,16.34,145908.0,3429.0,16094.0,3.88,2.41,11.03,13.70,163.0,0.21,1.01,72.21,OX40N,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SATIN,274.00,-22.47,72.0,H-SC,4.15,239724.0,-36560.0,179769.0,6.23,-13.23,74.99,51.83,126.0,-0.20,1.65,16.78,XY24,NTT,FINANCE
80,UJJIVANSFB,60.00,131.04,74.0,H-SC,16.34,145908.0,3429.0,16094.0,3.88,2.41,11.03,13.70,163.0,0.21,1.01,72.21,OX40N,NTT,BANKS
12,BANKINDIA,190.00,-16.77,74.0,H-MC,15.12,213459.0,33651.0,78766.0,3.65,18.72,36.90,62.52,88.0,0.43,1.47,59.18,XR,NTT,BANKS
37,INDIAMART,4810.62,-53.33,56.0,H-SC,7.02,130232.0,6896.0,124736.0,3.64,5.59,95.78,106.72,119.0,0.06,0.90,30.28,AR,ATH,MISC
54,QUESS,986.00,-46.56,45.0,X-SC,38.62,53108.0,-11898.0,162829.0,3.59,-18.30,306.60,232.18,198.0,-0.07,0.37,3.59,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJOOENG,143.10,-43.74,41.0,H-SC,18.48,85300.0,-17200.0,57799.0,-2.44,-16.78,67.76,39.61,114.0,-0.30,0.59,4.18,AR,ATH,MISC
20,COFFEEDAY,80.00,-45.67,48.0,L-SC,27.40,80947.0,-32602.0,72650.0,-2.39,-28.71,89.75,35.27,268.0,-0.45,0.56,98.12,XR,NTT,HOTELS
50,MASFIN,398.61,-20.51,44.0,H-SC,9.16,90285.0,-7695.0,29297.0,-2.35,-7.85,32.45,22.05,152.0,-0.26,0.62,31.24,XR,ATH,FINANCE
38,INDIGOPNTS,1408.00,91.41,32.0,M-SC,4.90,125550.0,-49009.0,49040.0,-1.95,-28.08,39.06,0.02,221.0,-1.00,0.87,9.21,OX40N,NTT,PAINTS
58,RELAXO,1176.00,-39.88,43.0,X-SC,4.69,81949.0,-63211.0,139141.0,-1.87,-43.55,169.79,52.31,136.0,-0.45,0.56,9.69,X40N,NTT,FOOTWEAR


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.0,41.94,48.0,M-MC,9.71,223843.0,-1119.0,168778.0,0.41,-0.50,75.40,74.53,192.0,-0.01,1.54,31.60,XY24,BTT,STEEL
36,IEX,219.0,-30.16,62.0,H-SC,17.20,199533.0,-421.0,97651.0,-0.01,-0.21,48.94,48.63,115.0,-0.00,1.38,11.32,XR,NTT,MISC
57,RECLTD,446.0,45.36,52.0,M-LC,5.69,204738.0,1568.0,40559.0,1.16,0.77,19.81,20.74,55.0,0.04,1.41,7.69,XY25,NTT,FINANCE
88,WIPRO,420.0,-13.79,47.0,M-LC,5.91,151150.0,205.0,109251.0,0.33,0.14,72.28,72.51,53.0,0.00,1.04,6.17,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,UJJIVANSFB,60.0,131.04,74.0,H-SC,16.34,145908.0,3429.0,16094.0,3.88,2.41,11.03,13.70,163.0,0.21,1.01,72.21,OX40N,NTT,BANKS
46,KANSAINER,340.0,-67.06,57.0,H-SC,2.54,227475.0,-42192.0,78524.0,-0.48,-15.65,34.52,13.47,138.0,-0.54,1.57,15.66,XY24,NTT,PAINTS
79,TTKPRESTIG,770.0,97.23,52.0,M-SC,2.06,84947.0,-15830.0,15919.0,0.55,-15.71,18.74,0.09,245.0,-0.99,0.59,11.60,OX40N,NTT,DURABLES
74,TATAELXSI,9161.0,-21.36,58.0,H-MC,7.30,106334.0,-25501.0,67724.0,0.95,-19.34,63.69,32.03,98.0,-0.38,0.73,19.10,OX40N,NTT,IT
68,SIS,528.0,2063.73,51.0,H-SC,3.18,87312.0,-23720.0,47332.0,-0.29,-21.36,54.21,21.26,156.0,-0.50,0.60,17.97,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,UJJIVANSFB,60.00,131.04,74.0,H-SC,16.34,145908.0,3429.0,16094.0,3.88,2.41,11.03,13.70,163.0,0.21,1.01,72.21,OX40N,NTT,BANKS
57,RECLTD,446.00,45.36,52.0,M-LC,5.69,204738.0,1568.0,40559.0,1.16,0.77,19.81,20.74,55.0,0.04,1.41,7.69,XY25,NTT,FINANCE
32,HINDZINC,730.22,29.16,51.0,M-LC,11.18,210040.0,4964.0,107603.0,-0.79,2.42,51.23,54.89,52.0,0.05,1.45,27.10,X5K,ATH,METALS
47,KPIGREEN,731.05,10.17,71.0,H-SC,5.26,131065.0,5768.0,53160.0,0.46,4.60,40.56,47.03,141.0,0.11,0.90,64.11,MH,ATH,POWER
81,UNIONBANK,163.00,-10.08,61.0,M-LC,9.28,163160.0,22320.0,22173.0,1.22,15.85,13.59,31.59,66.0,1.01,1.12,46.53,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,190.00,-16.77,74.0,H-MC,15.12,213459.0,33651.0,78766.0,3.65,18.72,36.90,62.52,88.0,0.43,1.47,59.18,XR,NTT,BANKS
80,UJJIVANSFB,60.00,131.04,74.0,H-SC,16.34,145908.0,3429.0,16094.0,3.88,2.41,11.03,13.70,163.0,0.21,1.01,72.21,OX40N,NTT,BANKS
87,WHIRLPOOL,2270.00,-38.27,65.0,M-SC,3.54,128064.0,13266.0,80770.0,1.56,11.56,63.07,81.92,223.0,0.16,0.88,51.61,XR,NTT,DURABLES
47,KPIGREEN,731.05,10.17,71.0,H-SC,5.26,131065.0,5768.0,53160.0,0.46,4.60,40.56,47.03,141.0,0.11,0.90,64.11,MH,ATH,POWER
37,INDIAMART,4810.62,-53.33,56.0,H-SC,7.02,130232.0,6896.0,124736.0,3.64,5.59,95.78,106.72,119.0,0.06,0.90,30.28,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TATAMOTORS,1065.00,-53.83,16.0,X-LC,15.22,146398.0,-130125.0,240195.0,-0.63,-47.06,164.07,39.81,54.0,-0.54,1.01,3.19,XY24,NTT,AUTO
45,JSWINFRA,342.00,-20.28,37.0,X-MC,5.42,199935.0,-412.0,30910.0,0.87,-0.21,15.46,15.23,178.0,-0.01,1.38,31.98,X40N,NTT,REALTY
24,DMART,5391.45,-16.50,38.0,X-LC,10.11,102168.0,-418.0,27228.0,1.02,-0.41,26.65,26.13,38.0,-0.02,0.70,25.06,X40N,ATH,FMCG
53,PGHH,17907.65,-30.55,38.0,X-MC,3.04,201180.0,360.0,67436.0,-1.03,0.18,33.52,33.76,80.0,0.01,1.39,5.39,X40,ATH,FMCG
52,PAGEIND,51769.93,-27.30,40.0,X-MC,6.90,82080.0,-1430.0,21464.0,0.04,-1.71,26.15,23.98,82.0,-0.07,0.57,4.01,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SIEMENS,4671.50,-1.76,51.0,H-LC,0.91,157840.0,-28255.0,75732.0,0.15,-15.18,47.98,25.51,15.0,-0.37,1.09,16.32,AR,ATH,ELECTRICAL
42,ITC,452.00,-36.23,67.0,X-LC,1.71,205380.0,5242.0,15650.0,0.77,2.62,7.62,10.44,4.0,0.33,1.42,8.33,X40,NTT,FMCG
29,HAVELLS,2069.16,-5.90,46.0,X-MC,2.91,313929.0,-14062.0,120580.0,-0.10,-4.29,38.41,32.48,92.0,-0.12,2.16,6.65,X40,ATH,ELECTRICAL
53,PGHH,17907.65,-30.55,38.0,X-MC,3.04,201180.0,360.0,67436.0,-1.03,0.18,33.52,33.76,80.0,0.01,1.39,5.39,X40,ATH,FMCG
86,VOLTAS,1530.00,2.56,58.0,X-MC,3.10,215070.0,23328.0,14431.0,-0.53,12.17,6.71,19.69,99.0,1.62,1.48,19.79,XY25,NTT,AC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COLPAL,3767.14,-5.20,42.0,X-MC,8.15,221510.0,-41855.0,155212.0,-1.09,-15.89,70.07,43.04,84.0,-0.27,1.53,2.84,XY25,ATH,FMCG
75,TATAMOTORS,1065.00,-53.83,16.0,X-LC,15.22,146398.0,-130125.0,240195.0,-0.63,-47.06,164.07,39.81,54.0,-0.54,1.01,3.19,XY24,NTT,AUTO
54,QUESS,986.00,-46.56,45.0,X-SC,38.62,53108.0,-11898.0,162829.0,3.59,-18.30,306.60,232.18,198.0,-0.07,0.37,3.59,XY24,NTT,MISC
10,BAJAJHFL,181.50,-14.48,45.0,X-MC,5.81,180220.0,-20286.0,117449.0,-0.29,-10.12,65.17,48.45,90.0,-0.17,1.24,3.92,X40N,ATH,FINANCE
52,PAGEIND,51769.93,-27.30,40.0,X-MC,6.90,82080.0,-1430.0,21464.0,0.04,-1.71,26.15,23.98,82.0,-0.07,0.57,4.01,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-46.56,45.0,X-SC,38.62,53108.0,-11898.0,162829.0,3.59,-18.30,306.60,232.18,198.0,-0.07,0.37,3.59,XY24,NTT,MISC
58,RELAXO,1176.00,-39.88,43.0,X-SC,4.69,81949.0,-63211.0,139141.0,-1.87,-43.55,169.79,52.31,136.0,-0.45,0.56,9.69,X40N,NTT,FOOTWEAR
52,PAGEIND,51769.93,-27.30,40.0,X-MC,6.90,82080.0,-1430.0,21464.0,0.04,-1.71,26.15,23.98,82.0,-0.07,0.57,4.01,X40,ATH,APPARELS
2,ABBOTINDIA,35195.00,-9.64,41.0,X-MC,3.21,89280.0,-1398.0,16303.0,-0.57,-1.54,18.26,16.44,101.0,-0.09,0.62,18.32,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-35.15,47.0,X-SC,4.35,96927.0,-31743.0,77038.0,0.23,-24.67,79.48,35.20,219.0,-0.41,0.67,11.25,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4389.97,-27.40,59.0,X-LC,12.05,289952.0,-56004.0,122708.0,0.70,-16.19,42.32,19.28,1.0,-0.46,2.00,7.19,X40,ATH,IT
40,INFY,2275.00,-17.80,59.0,X-LC,6.92,320011.0,7123.0,164550.0,-0.00,2.28,51.42,54.87,3.0,0.04,2.21,10.75,X40,BTT,IT
42,ITC,452.00,-36.23,67.0,X-LC,1.71,205380.0,5242.0,15650.0,0.77,2.62,7.62,10.44,4.0,0.33,1.42,8.33,X40,NTT,FMCG
85,VBL,671.64,-18.84,50.0,X-LC,9.81,292427.0,-23415.0,136066.0,-0.66,-7.41,46.53,35.67,5.0,-0.17,2.02,5.41,X40N,ATH,FMCG
1,ABB,7934.00,-39.87,53.0,H-LC,6.16,251208.0,-10411.0,129623.0,1.01,-3.98,51.60,45.57,7.0,-0.08,1.73,6.89,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7050.00,61.0,L-SC,7.31,78434.0,-15376.0,91995.0,-0.03,-16.39,117.29,81.67,269.0,-0.17,0.54,51.89,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,48.0,L-SC,27.40,80947.0,-32602.0,72650.0,-2.39,-28.71,89.75,35.27,268.0,-0.45,0.56,98.12,XR,NTT,HOTELS
50,MASFIN,398.61,-20.51,44.0,H-SC,9.16,90285.0,-7695.0,29297.0,-2.35,-7.85,32.45,22.05,152.0,-0.26,0.62,31.24,XR,ATH,FINANCE
87,WHIRLPOOL,2270.00,-38.27,65.0,M-SC,3.54,128064.0,13266.0,80770.0,1.56,11.56,63.07,81.92,223.0,0.16,0.88,51.61,XR,NTT,DURABLES
47,KPIGREEN,731.05,10.17,71.0,H-SC,5.26,131065.0,5768.0,53160.0,0.46,4.60,40.56,47.03,141.0,0.11,0.90,64.11,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SAMMAANCAP,326.00,-197.73,80.0,M-SC,26.58,168984.0,19764.0,124423.0,-0.26,13.24,73.63,96.62,208.0,0.16,1.16,82.88,XY25,NTT,FINANCE
5,ANGELONE,3033.00,19.39,75.0,X-SC,5.58,210871.0,19865.0,37830.0,2.27,10.40,17.94,30.21,157.0,0.53,1.45,33.45,X40N,NTT,FINANCE
69,SONACOMS,806.63,-31.93,72.0,M-SC,8.64,84656.0,-16503.0,56508.0,1.05,-16.31,66.75,39.54,202.0,-0.29,0.58,19.37,AR,ATH,AUTO
60,REPCOHOME,880.00,-55.29,72.0,H-SC,3.48,258262.0,-28243.0,285586.0,-0.02,-9.86,110.58,89.82,134.0,-0.10,1.78,36.64,XY24,NTT,FINANCE
47,KPIGREEN,731.05,10.17,71.0,H-SC,5.26,131065.0,5768.0,53160.0,0.46,4.60,40.56,47.03,141.0,0.11,0.90,64.11,MH,ATH,POWER


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.38
1,25,43.42
2,50,74.16


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.82
LC    31.96
MC    24.22
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.71
X40      16.18
XY25     11.99
XR       11.25
X40N     10.49
AR        7.75
OX40N     7.46
X200      1.75
X5K       1.45
SR        1.07
MH        0.90
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.22
X-LC    21.36
X-MC    18.55
M-SC    12.58
M-LC     5.02
X-SC     4.59
H-LC     4.57
H-MC     3.83
M-MC     1.54
L-SC     1.43
L-LC     1.01
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.30,-3.95,37.87
IT,12.44,-16.57,79.09
FINANCE,12.03,-8.16,57.19
MISC,6.83,-16.16,80.52
BANKS,6.59,-4.41,59.33
ELECTRICAL,5.75,-8.36,46.77
PAINTS,5.48,-16.29,33.63
INSURANCE,3.78,-0.87,38.48
AUTO,2.74,-44.41,104.68


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3263125.0,22
XR,1316113.0,14
AR,1165315.0,9
X40,812250.0,12
X40N,637505.0,9
OX40N,565094.0,10
XY25,442860.0,8
SR,265701.0,2
X5K,107603.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3336164.0,25
M-SC,1219264.0,15
X-MC,1187089.0,14
X-LC,1041604.0,13
X-SC,515760.0,6
H-MC,324443.0,3
M-LC,279586.0,4
H-LC,276249.0,3
L-SC,254470.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1146420.0      6
           AR         836811.0      5
           XR         776076.0      7
M-SC       XY24       693508.0      6
X-MC       XY24       581091.0      4
X-LC       X40        447216.0      5
           XY24       310373.0      2
X-MC       X40        287996.0      6
X-SC       X40N       275893.0      4
H-SC       SR         265701.0      2
           OX40N      257996.0      4
X-LC       X40N       213253.0      3
H-LC       AR         205355.0      2
H-MC       XY24       177953.0      1
X-MC       XY25       169643.0      2
M-MC       XY24       168778.0      1
L-SC       XR         164645.0      2
X-SC       XY24       162829.0      1
M-SC       OX40N      149549.0      4
X-MC       X40N       148359.0      2
M-SC       XR         128635.0      2
           XY25       124423.0      1
           AR         123149.0      2
M-LC       XR         109251.0      1
           X5K        107603.0      1
L-SC       OX40N       89825.0      1
H-MC       XR          78766.0      1
X-SC       X40         77038.0      1
H-LC       X200        70894.0      1
X-LC       XY25        70762.0      3
H-MC       OX40N       67724.0      1
L-MC       XR          58740.0      1
H-SC       MH          53160.0      1
M-LC       XY25        40559.0      1
L-LC       XY25        37473.0      1
M-LC       XY24        22173.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
